In [1]:
!pip install tqdm
!pip install "Pillow==9.5.0" #use a downgrade version of PIL
# Install PyTorch with CUDA support
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118  # For CUDA 11.8

!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
!pip install -U layoutparser
!pip install pytesseract
!pip install tensorboard



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.4) to /tmp/pip-install-c7rxfslu/detectron2_0d4764fa48f04f1fb3841189244d90f8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-c7rxfslu/detectron2_0d4764fa48f04f1fb3841189244d90f8
  Running command git checkout -q 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Preparing metadata (setup.py

# Initial Importing (from the publaynet dataset pretrained model) -> FOR INITIAL INFERENCE ONLY

In [5]:
import layoutparser as lp
model = lp.Detectron2LayoutModel(
            config_path ='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config', # In model catalog
            label_map   ={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}, # In model`label_map`
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
        )

# (NO NEEDED!! )Remap COCO -> as by DEFAULT the category_ids start from 1

As the model is pretrained on PubLayNet, we need to convert the annotations to the PubLayNet format -> labels have start with 0

In [ ]:
# # remap_coco.py
# import json

# # old→new mapping
# MAPPING = {1: 0, 2: 1, 3: 2, 4: 3}

# def remap_coco(input_json, output_json):
#     with open(input_json, 'r') as f:
#         data = json.load(f)

#     # remap categories
#     for cat in data['categories']:
#         cat['id'] = MAPPING[cat['id']]

#     # remap each annotation
#     for ann in data['annotations']:
#         ann['category_id'] = MAPPING[ann['category_id']]

#     # write out
#     with open(output_json, 'w') as f:
#         json.dump(data, f, indent=2)

# if __name__ == "__main__":
#     remap_coco("datasets/phase_1_training_minimal/annotations/train_phase_1_minimal_v2.json", "datasets/phase_1_training_minimal/annotations/train_phase_1_minimal_v2_remapped.json")
#     remap_coco("datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2.json",   "datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2_remapped.json")


# Register dataset

In [2]:
from detectron2.data.datasets import register_coco_instances
# ── Register your NEW 4-class splits ───────────────────────────────
register_coco_instances(
    "train_phase_1_minimal_v2_remapped", {}, 
    "/workspace/layoutparser/datasets/phase_1_training_minimal/annotations/train_phase_1_minimal_v2_remapped.json",
    "/workspace/layoutparser/datasets/phase_1_training_minimal/images"
)
register_coco_instances(
    "val_phase_1_minimal_v2_remapped",   {}, 
    "/workspace/layoutparser/datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2_remapped.json",
    "/workspace/layoutparser/datasets/phase_1_training_minimal/images"
)


# Just to check your environment

In [2]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# List all registered datasets
# print("Registered datasets:", DatasetCatalog.list())

metadata = MetadataCatalog.get("val_phase_1_minimal_v2_remapped")
print("Image root:", metadata.image_root)
print("Annotation file:", metadata.json_file)

Image root: ./datasets/phase_1_training_minimal/images
Annotation file: ./datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2_remapped.json


In [6]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os


cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))  # Base config

cfg.DATASETS.TRAIN = ("train_phase_1_minimal_v2_remapped",)
cfg.DATASETS.TEST = ("val_phase_1_minimal_v2_remapped",)

cfg.DATALOADER.NUM_WORKERS = 8 
# NOTE dont change weights -> USE PUBLAYNET PRETRAINED MODEL
cfg.MODEL.WEIGHTS = "pretrained_models/model_final.pth"  
cfg.SOLVER.LOG_PERIOD = 50 # Log every 50 iterations


cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025  # Lower learning rate for fine-tuning

# NOTE rule of thumb for count of iterations: num_images / IMS_PER_BATCH 
# if you have 40 images and IMS_PER_BATCH = 4, and you want to train for 20 epochs, then MAX_ITER = 40 / 4  = 200
"""
I have 64 images in train_phase_1_minimal_v2_remapped:
Assuming IMS_PER_BATCH = 4 and you want to train for 20 epochs:

- Iterations per epoch: 64 / 4 = 16
- For 20 epochs: 16 * 20 = 320
"""

cfg.SOLVER.MAX_ITER = 320    # Adjust based on your dataset size
cfg.SOLVER.STEPS = []         # No learning rate decay
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8

# Force CPU usage (no GPU on macOS)
cfg.MODEL.DEVICE = "cpu"  # This ensures the model runs on CPU

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 #IMPORTANT - number of classes

# Output directory -> NOTE u can change this
cfg.OUTPUT_DIR = "./training_results/training3_output_phase_1_minimal_detectron_ready"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


# Training (dont change)


In [7]:
from detectron2.engine import HookBase
from tqdm import tqdm
from detectron2.engine import DefaultTrainer
from detectron2.engine import HookBase
from tqdm import tqdm

class TQDMWithLossHook(HookBase):
    def before_train(self):
        self.pbar = tqdm(total=self.trainer.max_iter, desc="Training", unit="iter")

    def after_step(self):
        storage   = self.trainer.storage
        loss_dict = storage.latest()
        raw       = loss_dict.get("total_loss", None)

        # Unpack tuple if necessary
        if isinstance(raw, (tuple, list)):
            loss_value = raw[0]
        else:
            loss_value = raw

        # Now it's safe to float()
        if loss_value is not None:
            self.pbar.set_postfix(loss=float(loss_value))
        else:
            self.pbar.set_postfix(loss="N/A")

        self.pbar.update(1)

    def after_train(self):
        self.pbar.close()

# Then after you create your trainer:

trainer = DefaultTrainer(cfg)

# Add the tqdm hook
trainer.register_hooks([TQDMWithLossHook()])

# Start training
trainer.resume_or_load(resume=False) # <-- ensures a fresh load of the pretrained weights
trainer.train()


[05/05 19:34:45 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (7, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (7,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (24, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (24,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.


[05/05 19:34:45 d2.engine.train_loop]: Starting training from iteration 0


Training:   0%|          | 0/320 [00:00<?, ?iter/s]/opt/anaconda3/envs/layoutparser_experiment/lib/python3.11/site-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/opt/anaconda3/envs/layoutparser_experiment/lib/python3.11/site-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=d

[05/05 19:36:18 d2.utils.events]:  eta: 0:23:06  iter: 19  total_loss: 2.67  loss_cls: 1.501  loss_box_reg: 0.855  loss_rpn_cls: 0.09199  loss_rpn_loc: 0.1878  time: 4.5002  data_time: 0.0737  lr: 4.9953e-06  


...

Training: 100%|█████████▉| 319/320 [24:05<00:04,  4.22s/iter, loss=1.3]  

[05/05 19:58:55 d2.utils.events]:  eta: 0:00:00  iter: 319  total_loss: 1.123  loss_cls: 0.3845  loss_box_reg: 0.5273  loss_rpn_cls: 0.0376  loss_rpn_loc: 0.1787  time: 4.5215  data_time: 0.0012  lr: 7.992e-05  


Training: 100%|██████████| 320/320 [24:10<00:00,  4.17s/iter, loss=0.944]

[05/05 19:58:55 d2.engine.hooks]: Overall training speed: 318 iterations in 0:23:57 (4.5215 s / it)
[05/05 19:58:55 d2.engine.hooks]: Total training time: 0:23:58 (0:00:00 on hooks)
[05/05 19:58:55 d2.data.datasets.coco]: Loaded 16 images in COCO format from ./datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2.json
[05/05 19:58:55 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/05 19:58:55 d2.data.common]: Serializing 16 elements to byte tensors and concatenating them all ...
[05/05 19:58:55 d2.data.common]: Serialized dataset takes 0.01 MiB
WARNING [05/05 19:58:55 d2.engine.defaults]: No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


Training: 100%|██████████| 320/320 [24:10<00:00,  4.53s/iter, loss=0.944]


# Inference

### Step 1 : Build config same as training

In [8]:
import os, cv2
from detectron2.config        import get_cfg
from detectron2 import model_zoo
from detectron2.engine        import DefaultPredictor
from detectron2.data          import MetadataCatalog
from detectron2.utils.visualizer import Visualizer


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
))
cfg.MODEL.ROI_HEADS.NUM_CLASSES       = 4
cfg.MODEL.WEIGHTS                     = "training_results/training3_output_phase_1_minimal_detectron_ready/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
cfg.MODEL.DEVICE                      = "cpu"

### Step 2 Create predictor and metadata

In [9]:
predictor = DefaultPredictor(cfg)
metadata  = MetadataCatalog.get("val_phase_1_minimal_v2_remapped")


### Step 3 Inference and save

In [7]:
def infer_and_save(image_path, predictor, metadata, output_dir):
    im = cv2.imread(image_path)
    outputs = predictor(im)
    instances = outputs["instances"].to("cpu")

    v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=1.2)
    out = v.draw_instance_predictions(instances)

    os.makedirs(output_dir, exist_ok=True)
    base = os.path.splitext(os.path.basename(image_path))[0]
    save_path = os.path.join(output_dir, f"{base}_output.jpg")
    cv2.imwrite(save_path, out.get_image()[:, :, ::-1])
    print("Saved:", save_path)

# Usage
infer_and_save(
    "/Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/datasets/phase_1_training_minimal/images/920cb6ed__Appellant___Appellant_s_Bundle_of_Documents_Volume_1_Tab2_page_7.png",
    predictor,
    metadata,
    "/Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready"
)

Saved: /Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/920cb6ed__Appellant___Appellant_s_Bundle_of_Documents_Volume_1_Tab2_page_7_output.jpg


# Batch Inference 
1. Inferene the images available in the folder
2. Based on the json file

In [10]:
import json
# --- Paths (modify as needed) ---
images_dir = "/Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/datasets/phase_1_training_minimal/images"
json_path  = "/Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/datasets/phase_1_training_minimal/annotations/train_phase_1_minimal_v2_remapped.json"  # or val_phase_1_minimal_v2_remapped.json
output_dir = "/Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/training_images"

# 4) Prepare output directory for batch inference
os.makedirs(output_dir, exist_ok=True)

# 5) Load list of images from JSON
with open(json_path, 'r') as f:
    imgs = json.load(f)["images"]

# 6) Run inference & save visualizations
for img_info in imgs:
    img_file = img_info["file_name"] # get the image file name
    img_path = os.path.join(images_dir, img_file)
    if not os.path.exists(img_path):
        continue

    im = cv2.imread(img_path) # load image
    outputs = predictor(im) # run inference
    instances = outputs["instances"].to("cpu")

    # Save the visualized image
    v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=1.2)
    out = v.draw_instance_predictions(instances)
    save_name = os.path.splitext(img_file)[0] + "_output.jpg"
    save_path = os.path.join(output_dir, save_name)

    cv2.imwrite(save_path, out.get_image()[:, :, ::-1])
    print(f"Saved: {save_path}")


Saved: /Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/training_images/920cb6ed__Appellant___Appellant_s_Bundle_of_Documents_Volume_1_Tab2_page_7_output.jpg
Saved: /Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/training_images/78affaa4__Appellant_s_Bundle_of_Documents_Volume_1_Tab6_Tab7_page_8_output.jpg
Saved: /Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/training_images/dbd47ee1__Appellant___Appellant_s_Bundle_of_Documents_Volume_1_Tab2_page_19_output.jpg
Saved: /Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/training_images/bc80bfcb__Appellant_s_Bundle_of_Documents_Volume_1_Tab6_Tab1_page_2_output.jpg
Saved: /Users/doodledaron/Documents/F

# Evaluate the model

In [11]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [12]:
# Create evaluator and data loader for validation set
evaluator = COCOEvaluator("val_phase_1_minimal_v2_remapped", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "val_phase_1_minimal_v2_remapped")

# Run inference and evaluation
results = inference_on_dataset(trainer.model, val_loader, evaluator)
print(results)

# Save results to file
eval_output_dir = "/Users/doodledaron/Documents/Freelances/Leon/layoutparser_experiment/training_results/training3_output_phase_1_minimal_detectron_ready/evaluator"
os.makedirs(eval_output_dir, exist_ok=True)
with open(os.path.join(eval_output_dir, "eval_results.json"), "w") as f:
    json.dump(results, f, indent=2)

WARNING [05/05 20:05:54 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[05/05 20:05:54 d2.data.datasets.coco]: Loaded 16 images in COCO format from ./datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2.json
[05/05 20:05:54 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/05 20:05:54 d2.data.common]: Serializing 16 elements to byte tensors and concatenating them all ...
[05/05 20:05:54 d2.data.common]: Serialized dataset takes 0.01 MiB
[05/05 20:05:54 d2.evaluation.evaluator]: Start inference on 16 images
[05/05 20:06:02 d2.evaluation.evaluator]: Inference done 11/16. 0.5720 s / img. ETA=0:00:02
[05/05 20:06:25 d2.evaluation.evaluator]: Total inference time: 0:00:26.003141 (2.363922 s / img per device, on 1 devices)
[05/05 20:06:25 d2.evaluation.evaluator]:

# Issue: Class Imbalance
### Strategy:
1. Freeze the backbone : cfg.MODEL.BACKBONE.FREEZE_AT = 5  # Freeze all ResNet stages (0–4)
2. Enable RepeatFactorTrainingSampler (RFS)
3. Increase Iterations to 1000 and Tune LR Schedule
4. (Optional Bonus) 🔁 Use Augmentation (Drop-in)

### Configuration Setup

In [3]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("train_phase_1_minimal_v2_remapped",)
cfg.DATASETS.TEST  = ("val_phase_1_minimal_v2_remapped",)

# load the pretrained model weights from publaynet
cfg.MODEL.WEIGHTS = "pretrained_models/model_final.pth"  # from PubLayNet
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR       = 0.00025
cfg.SOLVER.MAX_ITER      = 1000 # # ~62.5 epochs at batch 4
"""
How it works :
- Training starts with your base learning rate ( cfg.SOLVER.BASE_LR = 0.00025 )
- At iteration 600, the learning rate will be multiplied by cfg.SOLVER.GAMMA = 0.1
- At iteration 800, it will be multiplied by cfg.SOLVER.GAMMA = 0.1 again

- Initial LR: 0.00025
- At iteration 600: LR becomes 0.000025
- At iteration 800: LR becomes 0.0000025
"""
cfg.SOLVER.STEPS         = (600, 800)
cfg.SOLVER.GAMMA         = 0.1

"""
- First 100 iterations: LR increases linearly from ~0 to 0.00025
- Helps stabilize early training
"""
cfg.SOLVER.WARMUP_ITERS  = 100
cfg.SOLVER.WARMUP_METHOD = "linear"

cfg.SOLVER.LOG_PERIOD    = 50
cfg.DATALOADER.NUM_WORKERS = 8  # use lower if CPU limited

"""
- Helps with class imbalance by oversampling rare categories
- Images with rare categories appear more frequently
"""
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler"

"""
- Setting REPEAT_THRESHOLD = 0.1 means any category that appears in less than 10% of your images will be oversampled
- This helps balance your training data where "Text" is dominant but "Figure", "Table", and "Title" are rare
"""
cfg.DATALOADER.REPEAT_THRESHOLD = 0.1

# freeze the first 5 layers of the backbone and train the remaining layers
# helps with faster training and less overfitting
# also helps with generalization as it reduces the risk of overfitting to the training data
cfg.MODEL.BACKBONE.FREEZE_AT = 5


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.DEVICE = "cpu"

cfg.OUTPUT_DIR = "./training_results/training3_output_phase_1_minimal_detectron_ready_solving_class_imbalance"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


###  Augmented Trainer

In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetMapper, build_detection_train_loader
import detectron2.data.transforms as T

class AugmentedTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        aug = [
            T.RandomRotation(angle=[-5, 5]),
            T.RandomBrightness(0.9, 1.1),
            T.RandomFlip(horizontal=True, vertical=False)
        ]
        mapper = DatasetMapper(cfg, is_train=True, augmentations=aug)
        return build_detection_train_loader(cfg, mapper=mapper)


###  Add Progress Bar Hook (Optional but Nice)

In [5]:
from detectron2.engine import HookBase
from tqdm import tqdm

class TQDMWithLossHook(HookBase):
    def before_train(self):
        self.pbar = tqdm(total=self.trainer.max_iter, desc="Training", unit="iter")
    def after_step(self):
        storage = self.trainer.storage
        loss_dict = storage.latest()
        loss = loss_dict.get("total_loss", None)
        if isinstance(loss, (tuple, list)):
            loss = loss[0]
        self.pbar.set_postfix(loss=float(loss) if loss else "N/A")
        self.pbar.update(1)
    def after_train(self):
        self.pbar.close()


In [6]:
trainer = AugmentedTrainer(cfg)
trainer.register_hooks([TQDMWithLossHook()])
trainer.resume_or_load(resume=False)
trainer.train()


[05/06 15:33:22 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (7, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (7,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (24, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (24,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.


[05/06 15:33:22 d2.engine.train_loop]: Starting training from iteration 0


Training:   0%|          | 0/1000 [00:00<?, ?iter/s]/opt/anaconda3/envs/layoutparser_experiment/lib/python3.11/site-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/opt/anaconda3/envs/layoutparser_experiment/lib/python3.11/site-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=

[05/06 16:17:22 d2.engine.hooks]: Overall training speed: 6 iterations in 0:06:46 (67.7240 s / it)
[05/06 16:17:22 d2.engine.hooks]: Total training time: 0:06:46 (0:00:00 on hooks)
[05/06 16:17:22 d2.utils.events]:  eta: 18:13:52  iter: 8  total_loss: 3.524  loss_cls: 1.894  loss_box_reg: 0.4738  loss_rpn_cls: 0.3745  loss_rpn_loc: 0.8135  time: 66.8219  data_time: 0.1868  lr: 1.7733e-05  


Training:   1%|          | 8/1000 [43:59<90:55:44, 329.98s/iter, loss=3.5] 


KeyboardInterrupt: 

# Configuration 1: Balanced Fine-Tuning with Augmentation

This notebook fine-tunes a PubLayNet-pretrained Faster R-CNN (R50-FPN) for document layout analysis under 3–4 hour A40 GPU runs.  
Key points:
- **Freeze** early backbone layers (up to stage 3) to retain pretrained features and reduce overfitting  
- **Oversample** rare classes (Table/Figure) via `RepeatFactorTrainingSampler` (threshold 0.2)  
- **Augment** with small rotations, brightness/contrast/saturation jitter, and lighting noise to simulate scanned PDFs  
- **Extend** RPN anchor aspect ratios to better cover wide titles and tall tables  
- **Hyperparameters**: batch 4, base LR=0.00025, max Iters=1000, LR drops at (600, 800), warmup=100 iters  


In [12]:
#%%
# 1) Imports & Base Config
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
)

# Datasets
cfg.DATASETS.TRAIN = ("train_phase_1_minimal_v2_remapped",)
cfg.DATASETS.TEST  = ("val_phase_1_minimal_v2_remapped",)

# Pretrained weights & classes
cfg.MODEL.WEIGHTS = "/workspace/layoutparser/pretrained_models/model_final.pth"  # PubLayNet checkpoint
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # for eval only

# Device
cfg.MODEL.DEVICE = "cuda"  # or "cpu" if no GPU


In [13]:
#%%
# 2) Imbalance sampler & backbone freezing
cfg.DATALOADER.SAMPLER_TRAIN    = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD = 0.2   # oversample images with rare classes

cfg.MODEL.BACKBONE.FREEZE_AT = 3  # freeze stem + res2 & res3


In [14]:
#%%
# 3) Anchor generator tweaks (wide/tall boxes)
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[32], [64], [128], [256], [512]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.2, 0.5, 1.0, 2.0, 5.0]]


In [15]:
#%%
# 4) Solver & LR schedule
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR       = 0.00025
cfg.SOLVER.MAX_ITER      = 1000  
cfg.SOLVER.STEPS         = (600, 800)
cfg.SOLVER.GAMMA         = 0.1
cfg.SOLVER.WARMUP_ITERS  = 100
cfg.SOLVER.WARMUP_METHOD = "linear"
cfg.SOLVER.LOG_PERIOD    = 50

cfg.DATALOADER.NUM_WORKERS = 4  # tune to your CPU cores


In [16]:
#%%
# 5) Augmented Trainer
from detectron2.engine import DefaultTrainer, HookBase
from detectron2.data import DatasetMapper, build_detection_train_loader
import detectron2.data.transforms as T
from tqdm import tqdm

class AugmentedTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        aug = [
            T.RandomRotation(angle=[-3, 3]),
            T.RandomBrightness(0.8, 1.2),
            T.RandomContrast(0.8, 1.2),
            T.RandomSaturation(0.8, 1.2),
            T.RandomLighting(0.2),
        ]
        mapper = DatasetMapper(cfg, is_train=True, augmentations=aug)
        return build_detection_train_loader(cfg, mapper=mapper)

class TQDMWithLossHook(HookBase):
    def before_train(self):
        self.pbar = tqdm(total=self.trainer.max_iter, desc="Train Config 1", unit="it")
    def after_step(self):
        loss = self.trainer.storage.latest().get("total_loss", 0)
        if isinstance(loss, (tuple, list)): loss = loss[0]
        self.pbar.set_postfix(loss=float(loss))
        self.pbar.update(1)
    def after_train(self):
        self.pbar.close()


In [17]:
#%%
# 6) Launch training
cfg.OUTPUT_DIR = "/workspace/layoutparser/training_results/config1_balanced_finetune"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = AugmentedTrainer(cfg)
trainer.register_hooks([TQDMWithLossHook()])
trainer.resume_or_load(resume=False)
trainer.train()


[05/07 11:33:46 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.0' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.1' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.2' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.3' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.4' to the model due to incompatible shapes: (3, 4) 

[05/07 11:33:47 d2.engine.train_loop]: Starting training from iteration 0



Train Config 1:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/usr/local/lib/python3.10/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/usr/local/lib/python3.10/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarr

[05/07 11:34:19 d2.utils.events]:  eta: 0:25:21  iter: 19  total_loss: 2.367  loss_cls: 0.6854  loss_box_reg: 0.001725  loss_rpn_cls: 0.7569  loss_rpn_loc: 0.8058  time: 1.5501  data_time: 0.0904  lr: 4.7702e-05  max_mem: 30651M


...


Train Config 1: 100%|█████████▉| 999/1000 [32:28<00:01,  1.58s/it, loss=0.853]

[05/07 12:06:18 d2.utils.events]:  eta: 0:00:00  iter: 999  total_loss: 0.7548  loss_cls: 0.2094  loss_box_reg: 0.1826  loss_rpn_cls: 0.02628  loss_rpn_loc: 0.3023  time: 1.9443  data_time: 0.1420  lr: 2.5e-06  max_mem: 30652M



Train Config 1: 100%|██████████| 1000/1000 [32:31<00:00,  1.98s/it, loss=0.633]

[05/07 12:06:18 d2.engine.hooks]: Overall training speed: 998 iterations in 0:32:20 (1.9443 s / it)
[05/07 12:06:18 d2.engine.hooks]: Total training time: 0:32:26 (0:00:05 on hooks)
[05/07 12:06:18 d2.data.datasets.coco]: Loaded 16 images in COCO format from /workspace/layoutparser/datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2_remapped.json
[05/07 12:06:18 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/07 12:06:18 d2.data.common]: Serializing 16 elements to byte tensors and concatenating them all ...
[05/07 12:06:18 d2.data.common]: Serialized dataset takes 0.01 MiB
WARNING [05/07 12:06:18 d2.engine.defaults]: No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


Train Config 1: 100%|██████████| 1000/1000 [32:31<00:00,  1.95s/it, loss=0.633]


In [25]:
#%%
import os, json
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# 1) Reload Config 1 (must match your training settings exactly)
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Datasets
cfg.DATASETS.TRAIN = ("train_phase_1_minimal_v2_remapped",)
cfg.DATASETS.TEST  = ("val_phase_1_minimal_v2_remapped",)

# Classes & Freeze
cfg.MODEL.ROI_HEADS.NUM_CLASSES   = 4
cfg.MODEL.BACKBONE.FREEZE_AT      = 3

# Imbalance sampler
cfg.DATALOADER.SAMPLER_TRAIN    = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD = 0.2

# Anchors
cfg.MODEL.ANCHOR_GENERATOR.SIZES         = [[32],[64],[128],[256],[512]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.2,0.5,1.0,2.0,5.0]]

# Solver (these aren’t needed for eval but safe to carry over)
cfg.SOLVER.IMS_PER_BATCH  = 4
cfg.SOLVER.BASE_LR        = 0.00025
cfg.SOLVER.MAX_ITER       = 1000
cfg.SOLVER.STEPS          = (600, 800)
cfg.SOLVER.GAMMA          = 0.1
cfg.SOLVER.WARMUP_ITERS   = 100
cfg.SOLVER.WARMUP_METHOD  = "linear"

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.DEVICE          = "cuda"

# 2) Point at your final model weights and lower the test threshold
cfg.MODEL.WEIGHTS               = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05  # low threshold for COCOEvaluator

# 3) Build & load the model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
model = trainer.model

# 4) Setup evaluator with explicit tasks
output_dir = os.path.join(cfg.OUTPUT_DIR, "evaluator")
os.makedirs(output_dir, exist_ok=True)
evaluator  = COCOEvaluator(
    "val_phase_1_minimal_v2_remapped",
    tasks=["bbox"],
    distributed=False,
    output_dir=output_dir
)
val_loader = build_detection_test_loader(cfg, "val_phase_1_minimal_v2_remapped")

# 5) Run evaluation
results = inference_on_dataset(model, val_loader, evaluator)
print("🏅 COCO eval results:\n", json.dumps(results, indent=2))



# 6) Save to JSON
# with open(os.path.join(output_dir, "evaluator/eval_results.json"), "w") as f:
#     json.dump(results, f, indent=2)
# print("✅ Saved eval_results.json to", output_dir)


[05/07 12:23:17 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

FileNotFoundError: [Errno 2] No such file or directory: 'output/metrics.json'

In [4]:
import os
import json
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# 1) Re-build the config exactly as in Config 1
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# ── Datasets ──────────────────────────────────
cfg.DATASETS.TRAIN = ("train_phase_1_minimal_v2_remapped",)
cfg.DATASETS.TEST  = ("val_phase_1_minimal_v2_remapped",)

# ── Model heads & thresholds ────────────────
cfg.MODEL.ROI_HEADS.NUM_CLASSES        = 4
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST  = 0.05   # low threshold so COCOEvaluator sees all boxes

# ── Device ──────────────────────────────────
cfg.MODEL.DEVICE = "cuda"  # or "cpu"

# ── Point to your actual weights directory ──
model_dir = "/workspace/layoutparser/training_results/config1_balanced_finetune"
cfg.OUTPUT_DIR     = model_dir
cfg.MODEL.WEIGHTS  = os.path.join(model_dir, "model_final.pth")

# 2) Build & load the trained model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
model = trainer.model

# 3) Create evaluator & validation dataloader
evaluator = COCOEvaluator(
    "val_phase_1_minimal_v2_remapped",
    cfg,
    distributed=False,
    output_dir=os.path.join(model_dir, "evaluator")
)
val_loader = build_detection_test_loader(cfg, "val_phase_1_minimal_v2_remapped")

# 4) Run inference & evaluation
results = inference_on_dataset(model, val_loader, evaluator)

print("🏅 COCO evaluation results:\n", json.dumps(results, indent=2))

# 5) Save results to JSON
eval_output_dir = os.path.join(model_dir, "evaluator")
os.makedirs(eval_output_dir, exist_ok=True)
with open(os.path.join(eval_output_dir, "eval_results.json"), "w") as f:
    json.dump(results, f, indent=2)

print(f"✅ Evaluation results saved to: {eval_output_dir}")


[05/07 12:52:04 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.weight' to the model due to incompatible shapes: (5, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.bias' to the model due to incompatible shapes: (5,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.weight' to the model due to incompatible shapes: (20, 256, 1, 1) in the checkpoint but (12, 256, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.bias' to the model due to incompatible shapes: (20,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.0' to the model due to inco

WARNING [05/07 12:52:04 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[05/07 12:52:04 d2.data.datasets.coco]: Loaded 16 images in COCO format from /workspace/layoutparser/datasets/phase_1_training_minimal/annotations/val_phase_1_minimal_v2_remapped.json
[05/07 12:52:04 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/07 12:52:04 d2.data.common]: Serializing 16 elements to byte tensors and concatenating them all ...
[05/07 12:52:04 d2.data.common]: Serialized dataset takes 0.01 MiB
[05/07 12:52:04 d2.evaluation.evaluator]: Start inference on 16 images


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/07 12:52:07 d2.evaluation.evaluator]: Inference done 11/16. 0.0949 s / img. ETA=0:00:00
[05/07 12:52:07 d2.evaluation.evaluator]: Total inference time: 0:00:01.270437 (0.115494 s / img per device, on 1 devices)
[05/07 12:52:07 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.086533 s / img per device, on 1 devices)
[05/07 12:52:07 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/07 12:52:07 d2.evaluation.coco_evaluation]: Saving results to /workspace/layoutparser/training_results/config1_balanced_finetune/evaluator/coco_instances_results.json
[05/07 12:52:07 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[05/07 12:52:07 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/07 12:52:07 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.00 seconds.
[05/07 12:52:07 d2.evaluation.fast_eval_api]:

Here’s a “best‐of‐both” recipe you can drop into your notebook—this config is tuned to:

Fix anchors so RPN sees wide/tall boxes

Balance rare classes (Table/Figure) without starving Titles

Freeze less of the backbone so mid‐level features can adapt

Tame augmentations so you don’t wash out small objects

Use weight decay & cosine LR to regularize and avoid overfitting

Clamp gradients for stability

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
)

# ── 1) Datasets ─────────────────────────────────────────────
cfg.DATASETS.TRAIN = ("train_phase_1_minimal_v2_remapped",)
cfg.DATASETS.TEST  = ("val_phase_1_minimal_v2_remapped",)

# ── 2) Model heads & inference thresh ──────────────────────
cfg.MODEL.ROI_HEADS.NUM_CLASSES        = 4
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST  = 0.05   # low for full AP calculation

# ── 3) Backbone freezing ───────────────────────────────────
# freeze only stem + first residual block (res2)
cfg.MODEL.BACKBONE.FREEZE_AT           = 2

# ── 4) Sampler for imbalance ──────────────────────────────
cfg.DATALOADER.SAMPLER_TRAIN           = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD        = 0.1   # oversample Table/Figure (& now Title at ~28%)

# ── 5) Anchor generator (one ratio list per FPN level) ────
cfg.MODEL.ANCHOR_GENERATOR.SIZES       = [[32], [64], [128], [256], [512]]
ratios = [0.2, 0.5, 1.0, 2.0, 5.0]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [ratios] * len(cfg.MODEL.ANCHOR_GENERATOR.SIZES)

# ── 6) Solver & LR schedule ────────────────────────────────
cfg.SOLVER.IMS_PER_BATCH               = 4
cfg.SOLVER.BASE_LR                     = 1e-4      # slightly lower than before
cfg.SOLVER.MAX_ITER                    = 1500      # ~ a few epochs on 63 imgs
cfg.SOLVER.LR_SCHEDULER_NAME           = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS                = 100
cfg.SOLVER.WARMUP_METHOD               = "linear"
cfg.SOLVER.COSINE_END_LR               = 1e-6
# no step LR—cosine covers full run

# ── 7) Regularization ──────────────────────────────────────
cfg.SOLVER.WEIGHT_DECAY      = 1e-4
cfg.SOLVER.WEIGHT_DECAY_NORM = 0.0

# Gradient clipping to stabilize small-batch training:
cfg.SOLVER.CLIP_GRADIENTS.ENABLED    = True
cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE  = "norm"
cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 1.0


# ── 8) Data augmentation ───────────────────────────────────
from detectron2.data import DatasetMapper, build_detection_train_loader
import detectron2.data.transforms as T
class CleanAugTrainer(cfg.__class__): pass  # dummy

class CleanAugTrainer(cfg.__class__): pass  # for type-hint
from detectron2.engine import DefaultTrainer, HookBase
class CleanAugTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        aug = [
            T.RandomRotation(angle=[-2, 2]),         # slight skew
            T.RandomBrightness(0.9, 1.1),            # mild
            T.RandomContrast(0.9, 1.1),
        ]
        mapper = DatasetMapper(cfg, is_train=True, augmentations=aug)
        return build_detection_train_loader(cfg, mapper=mapper)

# ── 9) Device & output ────────────────────────────────────
cfg.MODEL.DEVICE = "cuda"
cfg.OUTPUT_DIR    = "/workspace/layoutparser/training_results/config1_balanced_finetuneconfig_improved"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.WEIGHTS = "/workspace/layoutparser/pretrained_models/model_final.pth"  # keep PubLayNet init

# ── 10) Launch ─────────────────────────────────────────────
from detectron2.engine import DefaultTrainer
trainer = CleanAugTrainer(cfg)
trainer.register_hooks([HookBase()])  # you can register a TQDM hook here
trainer.resume_or_load(resume=False)
trainer.train()


[05/07 13:15:35 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.0' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.1' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.2' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.3' to the model due to incompatible shapes: (3, 4) in the checkpoint but (5, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.anchor_generator.cell_anchors.4' to the model due to incompatible shapes: (3, 4) 

[05/07 13:15:35 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/usr/local/lib/python3.10/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/usr/local/lib/python3.10/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the lis

[05/07 13:16:40 d2.utils.events]:  eta: 0:53:18  iter: 19  total_loss: 2.771  loss_cls: 1.292  loss_box_reg: 0.001128  loss_rpn_cls: 0.7579  loss_rpn_loc: 0.8116  time: 3.0982  data_time: 1.3641  lr: 1.8873e-05  max_mem: 35187M
[05/07 13:17:18 d2.utils.events]:  eta: 0:44:24  iter: 39  total_loss: 2.453  loss_cls: 0.9663  loss_box_reg: 0.001014  loss_rpn_cls: 0.743  loss_rpn_loc: 0.7707  time: 2.4637  data_time: 0.1333  lr: 3.8635e-05  max_mem: 35187M
[05/07 13:18:04 d2.utils.events]:  eta: 0:43:37  iter: 59  total_loss: 1.798  loss_cls: 0.3496  loss_box_reg: 0.001053  loss_rpn_cls: 0.7052  loss_rpn_loc: 0.744  time: 2.4001  data_time: 0.3726  lr: 5.8396e-05  max_mem: 35187M
[05/07 13:18:39 d2.utils.events]:  eta: 0:41:53  iter: 79  total_loss: 1.433  loss_cls: 0.06211  loss_box_reg: 0.0009944  loss_rpn_cls: 0.653  loss_rpn_loc: 0.7005  time: 2.2410  data_time: 0.0305  lr: 7.8158e-05  max_mem: 35187M
[05/07 13:19:14 d2.utils.events]:  eta: 0:41:05  iter: 99  total_loss: 1.23  loss_cls: